In [2]:
import pandas as pd
import fasttext
from tqdm.notebook import tqdm

In [3]:
# # Path to the pre-trained model
# # file in local directory (jupyter notebook)
# model_path = './cc.ko.300.bin'

# # Loading the pre-trained model
# pretrained_model = fasttext.load_model(model_path)

In [120]:
# load dataframe
df = pd.read_csv('./newly_reduced_bungae_df.csv', lineterminator='\n') # 이걸로 오류 해결
df['id'] = df['id'].astype(str)
df['cat_id'] = df['cat_id'].astype(str)
df['maincat_id'] = df['maincat_id'].astype(str)
df['midcat_id'] = df['midcat_id'].astype(str)
df.head()

,id,name,info,cat_id,tag,maincat_id,midcat_id
0,202546050,알토라쿤잠바,알토 라쿤잠바입니다 55사이즈네요 모자털 탈부착가능합니다,310300200,"알토,라쿤,잠바",310,310300
1,175146200,코스 오라리맛 푸퍼 다크브라운 40 판매합니다 (코스 다운 패디드 푸퍼),새상품 판매해요 택포,310300200,"코스푸퍼,코스,푸퍼",310,310300
2,213905890,[S] 커버낫 숏패딩,세탁완료! 깨끗하게 입었습니다 지퍼가 잘 됐다 안됐다 그럽니다;; 저는 똑딱이로 ...,310300200,"커버낫,숏,다운,오리털,패딩",310,310300
3,216998343,더 틸버리 구스패딩,55/한겨울 코트속에 이른봄은 외투로 활용도 높은 구스패딩입니다 컨디션최상~,310300200,더틸버리,310,310300
4,225515936,Sinoon 시눈 패딩 ELIN PATCHED PADDING,택 달려있어요 잠깐 외출할 때 몇번 입었구 하자 없습니다~!,310300200,"시눈,sinoon,패딩,프레클,스푸닝",310,310300


In [125]:
# split data and save as txt file
from sklearn.model_selection import train_test_split

def split_train_test_data(df):
    # -- train, test split --
    train_x, test_x, train_y, test_y = train_test_split(df['name'],
                                                        df['midcat_id'].values, # mid  
#                                                         df['cat_id'].values, # sub  
                                                        random_state=42, 
                                                        test_size=.2)
    # -- train_df, test_df split --
    train_df, test_df = df.loc[train_x.index], df.loc[test_x.index]
    
    # -- resetting index -- 
    train_df = train_df.reset_index(drop=True)
    test_df = test_df.reset_index(drop=True)

    return train_df, test_df

def label_and_save_as_txt(train_df, test_df):
    # Save train_df to a text file
    train_df['label'] = '__label__' + train_df['midcat_id'] # mid
#     train_df['label'] = '__label__' + train_df['cat_id'] # sub
    
    train_df = train_df[['label', 'name']]
    train_df.to_csv('train_fasttext.txt', sep='\t', index=False)

    # Save test_df to a text file
    test_df['label'] = '__label__' + test_df['midcat_id'] # mid
#     test_df['label'] = '__label__' + test_df['cat_id'] # sub
    test_df = test_df[['label', 'name']]
    test_df.to_csv('test_fasttext.txt', sep='\t', index=False)
    
    return train_df, test_df

def split_and_save_txt_file(df):
    train_df, test_df = split_train_test_data(df)
    train_df, test_df = label_and_save_as_txt(train_df, test_df)
    return train_df, test_df

In [126]:
import fasttext
train_df, test_df = split_and_save_txt_file(df)

model = fasttext.train_supervised(input='./train_fasttext.txt',
                                  wordNgrams=2,
                                  epoch=15,
                                  lr=0.34,
                                  loss='ova')

Read 0M words
Number of words:  80099
Number of labels: 45
Progress: 100.0% words/sec/thread:  242300 lr:  0.000000 avg.loss:  0.614554 ETA:   0h 0m 0s


In [128]:
# predictions
def get_predictions(model, test_df):
    for idx in tqdm(range(len(test_df))):
        prd_name = test_df.loc[idx, 'name'].replace('\n', '') # remove newline characters
        predicted_labels = model.predict(prd_name, k=3)[0] # predicting top 3 labels
        test_df.at[idx, 'predicted_labels'] = ' '.join(predicted_labels)
    return test_df

# accuracy score
def get_accuracy_score(test_df):
    for idx in tqdm(range(len(test_df))):
        predicted_labels = test_df.loc[idx, 'predicted_labels'].split(' ')
        label = test_df.loc[idx, 'label']
        if label in predicted_labels:
            test_df.loc[idx, 'accuracy'] = 1
        else:
            test_df.loc[idx, 'accuracy'] = 0
            
    return test_df['accuracy'].sum() / len(test_df)

def get_predictions_and_accuracy_score(model, test_df):
    test_df = get_predictions(model, test_df)
    accuracy_score = get_accuracy_score(test_df)
    return accuracy_score

In [129]:
get_predictions_and_accuracy_score(model, test_df)

  0%|          | 0/21850 [00:00<?, ?it/s]

  0%|          | 0/21850 [00:07<?, ?it/s]

0.8209610983981693